In [9]:
using VectorSphericalHarmonics
using SpecialFunctions
using LinearAlgebra
using GR
using ProgressMeter

l_num = 284
lambda = 1594.79581941464
c = 299792458

function E_TE(r, theta, phi, lambda, l, m, n, R)
    xr0 = 2*pi*r*1e3/lambda
    xr = xr0*n
    xR0 = 2*pi*R*1e3/lambda
    xR = xR0*n
    if r<=R
        fr = sqrt(pi/(2*xr))*besselj(l+1/2, xr)
    else
        fr = sqrt(pi/(2*xr))*besselj(l+1/2, xR)*besselh(l+1/2, 2, xr0)/besselh(l+1/2, 2, xR0)
    end
    fo = -1im*r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 0, theta, phi)
    return fr*fo
end

function sphericalbesselj_d(l, x)
    return sqrt(pi*x/2)*(besselj(l+1/2, x)/(2*x)+besselj(l-1/2, x)/2-besselj(l+3/2, x)/2)
end

function sphericalbesselh_d(l, x)
    return sqrt(pi*x/2)*(besselh(l+1/2, 2, x)/(2*x)+besselh(l-1/2, 2, x)/2-besselh(l+3/2, 2, x)/2)
end

function H_TE(r, theta, phi, lambda, l, m, n, R)
    xr0 = 2*pi*r*1e3/lambda
    xr = xr0*n
    xR0 = 2*pi*R*1e3/lambda
    xR = xR0*n
    if r<=R
        f1 = l*(l+1)*sqrt(pi*xr/2)*besselj(l+1/2, xr)*vshbasis(Hansen(), Polar(), l, m, -1, theta, phi)/xr^2
        f2 = sphericalbesselj_d(l, xr)*r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 1, theta, phi)/xr
        f_tol = (1im*n/c)*(f1+f2)
    else
        f1 = l*(l+1)*sqrt(pi*xr0/2)*besselh(l+1/2, 2, xr0)*vshbasis(Hansen(), Polar(), l, m, -1, theta, phi)/xr0^2
        f2 = sphericalbesselh_d(l, xr0)*r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 1, theta, phi)/xr0
        f_tol = (1im/(n*c))*sqrt(n)*(besselj(l+1/2, xR)/besselh(l+1/2, 2, xR0))*(f1+f2)
    end
    return f_tol
end

function E_TM(r, theta, phi, lambda, l, m, n, R)
    xr0 = 2*pi*r*1e3/lambda
    xr = xr0*n
    xR0 = 2*pi*R*1e3/lambda
    xR = xR0*n
    if r<=R
        f1 = l*(l+1)*sqrt(pi*xr/2)*besselj(l+1/2, xr)*vshbasis(Hansen(), Polar(), l, m, -1, theta, phi)/xr^2
        f2 = sphericalbesselj_d(l, xr)*r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 1, theta, phi)/xr
        f_tol = f1+f2
    else
        f1 = l*(l+1)*sqrt(pi*xr0/2)*besselh(l+1/2, 2, xr0)*vshbasis(Hansen(), Polar(), l, m, -1, theta, phi)/xr0^2
        f2 = sphericalbesselh_d(l, xr0)*r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 1, theta, phi)/xr0
        f_tol = sqrt(n)*(besselj(l+1/2, xR)/besselh(l+1/2, 2, xR0))*(f1+f2)
    end
    return f_tol
end

function M_TM(r, theta, phi, lambda, l, m, n, R)
    xr0 = 2*pi*r*1e3/lambda
    xr = xr0*n
    xR0 = 2*pi*R*1e3/lambda
    xR = xR0*n
    if r<=R
        fr = sqrt(pi/(2*xr))*besselj(l+1/2, xr)
    else
        fr = sqrt(pi/(2*xr))*besselj(l+1/2, xR)*besselh(l+1/2, 2, xr0)/besselh(l+1/2, 2, xR0)
    end
    fo = r*sqrt(l*(l+1))*vshbasis(Hansen(), Polar(), l, m, 0, theta, phi)
    return n*fr*fo/c
end

M_TM (generic function with 1 method)

In [3]:
function field(r, theta, lambda, l_num, m_num, n, R, mode, field)
    field_type = ["E_r", "E_theta", "E_phi", "E", "E^2", "H_r", "H_theta", "H_phi", "H", "H^2"]
    type = findfirst(x -> x==field, field_type)
    if mode == "TE"
        if 1 <= type <= 3
            return norm(E_TE(r, theta, 0, lambda, l_num, m_num, n, R)[type])
        elseif type == 4
            return norm(E_TE(r, theta, 0, lambda, l_num, m_num, n, R))
        elseif type == 5
            return norm(E_TE(r, theta, 0, lambda, l_num, m_num, n, R))^2
        elseif 6 <= type <= 8
            return norm(H_TE(r, theta, 0, lambda, l_num, m_num, n, R)[type-5])
        elseif type == 9
            return norm(H_TE(r, theta, 0, lambda, l_num, m_num, n, R))
        elseif type == 10
            return norm(H_TE(r, theta, 0, lambda, l_num, m_num, n, R))^2
        else
            println("Error: field_type of $filed doesn't exist")
            return nothing
        end
    elseif mode == "TM"
        if 1 <= type <= 3
            return norm(E_TM(r, theta, 0, lambda, l_num, m_num, n, R)[type])
        elseif type == 4
            return norm(E_TM(r, theta, 0, lambda, l_num, m_num, n, R))
        elseif type == 5
            return norm(E_TM(r, theta, 0, lambda, l_num, m_num, n, R))^2
        elseif 6 <= type <= 8
            return norm(H_TM(r, theta, 0, lambda, l_num, m_num, n, R)[type-5])
        elseif type == 9
            return norm(H_TM(r, theta, 0, lambda, l_num, m_num, n, R))
        elseif type == 10
            return norm(H_TM(r, theta, 0, lambda, l_num, m_num, n, R))^2
        else
            println("Error: field_type of $filed doesn't exist")
            return nothing
        end
    else
        println("Error: mode of $mode doesn't exist")
        return nothing
    end
end

function whole(θ, polardata)
    nq = length(θ)
    θ = collect(θ)
    for (i, v) in enumerate(reverse(θ)[2:nq])
        append!(θ, pi-v)
        polardata = hcat(polardata, polardata[:, nq-i])
    end
    for (i, v) in enumerate(reverse(θ)[2:2*nq-1])
        append!(θ, 2*pi-v)
        polardata = hcat(polardata, polardata[:, 2*nq-1-i])
    end
    return θ, polardata
end

whole (generic function with 1 method)

In [6]:
dsp = [50, 100]
R=50
R_region = 60
θ = range(0, stop=pi/2, length=dsp[2])

function radius_array(len, R, R_region)
    if R_region <= R
        r = sort([density(x, R, R_region) for x = range(0, stop=R_region, length=len)])
    else
        l1 = floor(Int, len*R/R_region)
        range_r1 = range(0, stop=R, length=l1)
        p = (R_region-R)/(len-l1)
        range_r2 = R+p:p:R_region
        range_r = vcat(range_r1, range_r2)
        r = sort([density(x, R, R_region) for x = range_r])
    end
    return r
end

r = radius_array(dsp[1], R, R_region);
# r = sort([density(x, 50, 60) for x = range_r]);
polardata = zeros((dsp[1], dsp[2]))
for i=1:length(r)
    for j=1:length(θ)
        pd = field(r[i], θ[j], lambda, l_num, l_num, 1.5, 50, "TE", "E_theta")
        polardata[i, j] = isnan(pd) ? 0 : pd
    end
end

θ, polardata = whole(θ, polardata);